In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [10]:
device = "cpu"

In [11]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [12]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [13]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = optim.SGD(model.parameters(), lr=2e-2)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step()

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().cpu().numpy().item()))

epoch:  0, loss: 0.2982991635799408
epoch:  1, loss: 0.2728068232536316
epoch:  2, loss: 0.24976131319999695
epoch:  3, loss: 0.22895631194114685
epoch:  4, loss: 0.2102551907300949
epoch:  5, loss: 0.19349995255470276
epoch:  6, loss: 0.17844733595848083
epoch:  7, loss: 0.164863720536232
epoch:  8, loss: 0.15257015824317932
epoch:  9, loss: 0.14142927527427673
epoch:  10, loss: 0.13132351636886597
epoch:  11, loss: 0.12215439975261688
epoch:  12, loss: 0.11383365839719772
epoch:  13, loss: 0.10628622025251389
epoch:  14, loss: 0.09944295883178711
epoch:  15, loss: 0.09324000030755997
epoch:  16, loss: 0.08762012422084808
epoch:  17, loss: 0.08252923935651779
epoch:  18, loss: 0.07791908830404282
epoch:  19, loss: 0.07374623417854309
epoch:  20, loss: 0.0699709802865982
epoch:  21, loss: 0.06655686348676682
epoch:  22, loss: 0.06347063183784485
epoch:  23, loss: 0.06068222597241402
epoch:  24, loss: 0.058163899928331375
epoch:  25, loss: 0.055890753865242004
epoch:  26, loss: 0.053839

In [14]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = -3232.1519949179137
Test metrics:  R2 = -2950.4635092701596


In [15]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=1e-2)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step()

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().cpu().numpy().item()))

epoch:  0, loss: 0.1731976866722107
epoch:  1, loss: 0.12661141157150269
epoch:  2, loss: 0.09270784258842468
epoch:  3, loss: 0.06961290538311005
epoch:  4, loss: 0.05111697316169739
epoch:  5, loss: 0.038837119936943054
epoch:  6, loss: 0.035239897668361664
epoch:  7, loss: 0.041287388652563095
epoch:  8, loss: 0.04841260984539986
epoch:  9, loss: 0.04987475275993347
epoch:  10, loss: 0.046801935881376266
epoch:  11, loss: 0.042317334562540054
epoch:  12, loss: 0.0384860523045063
epoch:  13, loss: 0.03611890971660614
epoch:  14, loss: 0.035180795937776566
epoch:  15, loss: 0.03522069752216339
epoch:  16, loss: 0.03570188954472542
epoch:  17, loss: 0.03618163242936134
epoch:  18, loss: 0.03639134764671326
epoch:  19, loss: 0.03620212897658348
epoch:  20, loss: 0.03565441444516182
epoch:  21, loss: 0.034885093569755554
epoch:  22, loss: 0.03408835083246231
epoch:  23, loss: 0.033459145575761795
epoch:  24, loss: 0.03313704952597618
epoch:  25, loss: 0.033159054815769196
epoch:  26, los

In [16]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7200987036630031
Test metrics:  R2 = 0.6411664841464888
